### 切分test_ffm.csv和test.csv，分开预测输出submission.csv后合并
* 新建test文件夹，并将转换好的完整.csv存放进去
* 新建test_ffm文件夹，并将转换好的完整test_ffm.csv存放进去
* 新建submission文件夹
* 我写的是100W行切分，切出来多少份看具体情况，假设切除来的是N份

In [ ]:
from csv import DictWriter
import pandas as pd
import os

In [ ]:
def mkSubFile_ffm(lines, srcName, sub):
    [des_filename, extname] = os.path.splitext(srcName)
    filename = des_filename + '_' + str(sub) + extname
    print('make file: %s' % filename)
    fout = open(filename, 'w')
    try:
        fout.writelines(lines)
        return sub + 1
    finally:
        fout.close()

In [ ]:
def splitByLineCount_ffm(filename, count):
    fin = open(filename, 'r')
    try:
        buf = []
        sub = 1
        for line in fin:
            buf.append(line)
            if len(buf) == count:
                sub = mkSubFile(buf, head, filename, sub)
                buf = []
        if len(buf) != 0:
            sub = mkSubFile(buf, head, filename, sub)
    finally:
        fin.close()

In [ ]:
def mkSubFile(lines, head, srcName, sub):
    [des_filename, extname] = os.path.splitext(srcName)
    filename = des_filename + '_' + str(sub) + extname
    print('make file: %s' % filename)
    fout = open(filename, 'w')
    try:
        fout.writelines([head])
        fout.writelines(lines)
        return sub + 1
    finally:
        fout.close()

In [ ]:
def splitByLineCount(filename, count):
    fin = open(filename, 'r')
    try:
        head = fin.readline()
        buf = []
        sub = 1
        for line in fin:
            buf.append(line)
            if len(buf) == count:
                sub = mkSubFile(buf, head, filename, sub)
                buf = []
        if len(buf) != 0:
            sub = mkSubFile(buf, head, filename, sub)
    finally:
        fin.close()

In [ ]:
path = 'data1'

test_name = 'test'
test_ffm_name = 'test_ffm'
splitByLineCount_ffm('%s/test_ffm/%s.csv'%(path,test_ffm_name),1000000)
splitByLineCount('%s/test/%s.csv'%(path,test_name),1000000)

### 循环预测
* 新建submission文件夹放批量预测的结果
* 预测的end=上面切出的份数，N

In [ ]:
import pandas as pd
from pandas import get_dummies
import lightgbm as lgb
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy import sparse
import numpy as np
import os
import gc
import xlearn as xl
import time

In [ ]:
path = 'data1'
test_name = 'test'
train_ffm_name = 'train_ffm.csv'
vali_ffm_name = 'vali_ffm.csv'
test_ffm_name = 'test_ffm'
submission_name = 'submission'

model_name = path+"/model.out"
output_name = path+"/output.txt"

In [ ]:
model_name = path+"/model.out"
output_name = path+"/submission/output.txt"
beg = 1
end = 12      # test切分的份数
for i in range(beg,end+1):        # 
    ffm_model = xl.create_ffm()
    ffm_model.setSigmoid()
    s_1 = time.time()
    ffm_model.setTest('%s/test_ffm/%s_%d.csv'%(path,test_ffm_name,i))
    ffm_model.predict(model_name,output_name)
    sub = pd.DataFrame()
    test_df = pd.read_csv('%s/test/%s_%d.csv'%(path,test_name,i))
    sub['aid']=test_df['aid']
    sub['uid']=test_df['uid']
    sub['score'] = np.loadtxt(output_name)
    sub.to_csv('%s/submission/%s_%d.csv'%(path,submission_name,i),index=False)
    if os.path.exists(output_name):
        os.remove(output_name)
    del test_df
    del ffm_model
    del sub
    gc.collect()
    s_2 = time.time()
    print('%s_%d.csv use time : %d'%(test_name,i,(s_2-s_1)))

### 结合N份submission_*.csv子文件
* 份数为N,这里是12

In [ ]:
import pandas as pd
import numpy as np
import gc
import time

In [ ]:
def combineFile(filename, num):
    for i in range(1,num+1):
        try:
            load_filename = 'data1/submission/submission_%d.csv'%i
            fin = open(load_filename,'r')
            fout = open(filename,'a')
            head = fin.readline()
            if i == 1:
                fout.writelines([head])
            buf = []
            for line in fin:
                buf.append(line)
            fout.writelines(buf)
        finally:
            fin.close()
            fout.close()

In [ ]:
N = 12
combineFile('data1/submission.csv',N)